In [ ]:
import pandas as pd
df = pd.read_csv("../data/processed data/processed_data.csv")
df.head()

,date,air_temperature_mean,dewpoint_mean,rltv_hum_mean,air_temperature_min,rltv_hum_min,air_temperature_max,prcp_count,mean_consumption,median_consumption,std_consumption,day,holiday
0,2011-11-23,5.925000,5.208333,95.250000,2.4,82.3,10.6,0,7.063833,6.2460,5.373299,2,0
1,2011-11-24,9.670833,8.041667,90.112500,6.1,74.5,13.6,2,8.426043,7.4890,6.219456,3,0
2,2011-11-25,10.541667,6.891667,78.779167,6.1,59.8,13.0,2,9.350267,8.7385,6.377867,4,0
3,2011-11-26,9.416667,6.333333,81.745833,4.3,66.9,12.9,0,10.255256,9.4770,6.689567,5,0
4,2011-11-27,9.904167,5.004167,73.425000,0.5,46.8,13.9,0,10.946154,9.8030,7.351755,6,0


In [1]:
df['date'] = pd.to_datetime(df['date'])

NameError: name 'pd' is not defined

# Data Preprocessing

In [ ]:
import numpy as np

class CyclicDateEncoder:
    def __init__(self, date_column='date'):
        self.date_column = date_column

    def _cyclic_encode(self, values, max_value):
        """
        Generates sine and cosine encoding for cyclic values.
        """
        sin_values = np.sin(2 * np.pi * values / max_value)
        cos_values = np.cos(2 * np.pi * values / max_value)
        return sin_values, cos_values

    def add_cyclic_features(self, df):
        """
        Adds cyclic encoding columns for month, day of month, and day of week.
        """
        # Extract date parts
        df['day_of_month'] = df[self.date_column].dt.day
        df['month'] = df[self.date_column].dt.month
        df['day_of_week'] = df[self.date_column].dt.weekday

        # Cyclic encoding
        df['month_sin'], df['month_cos'] = self._cyclic_encode(df['month'], 12)
        df['day_of_week_sin'], df['day_of_week_cos'] = self._cyclic_encode(df['day_of_week'], 7)

        # Optionally, drop the intermediate columns if you only want sin/cos
        df.drop(columns=['month', 'day_of_month', 'day_of_week'], inplace=True)

        return df

encoder = CyclicDateEncoder(date_column='date')
df = encoder.add_cyclic_features(df)

df.head()

,date,air_temperature_mean,dewpoint_mean,rltv_hum_mean,air_temperature_min,rltv_hum_min,air_temperature_max,prcp_count,mean_consumption,median_consumption,std_consumption,day,holiday,month_sin,month_cos,day_of_week_sin,day_of_week_cos
0,2011-11-23,5.925000,5.208333,95.250000,2.4,82.3,10.6,0,7.063833,6.2460,5.373299,2,0,-0.5,0.866025,0.974928,-0.222521
1,2011-11-24,9.670833,8.041667,90.112500,6.1,74.5,13.6,2,8.426043,7.4890,6.219456,3,0,-0.5,0.866025,0.433884,-0.900969
2,2011-11-25,10.541667,6.891667,78.779167,6.1,59.8,13.0,2,9.350267,8.7385,6.377867,4,0,-0.5,0.866025,-0.433884,-0.900969
3,2011-11-26,9.416667,6.333333,81.745833,4.3,66.9,12.9,0,10.255256,9.4770,6.689567,5,0,-0.5,0.866025,-0.974928,-0.222521
4,2011-11-27,9.904167,5.004167,73.425000,0.5,46.8,13.9,0,10.946154,9.8030,7.351755,6,0,-0.5,0.866025,-0.781831,0.623490


In [ ]:
drop_cols = ["day", "date"]
df.drop(columns=['date'], inplace=True)

In [ ]:
df.columns

Index(['air_temperature_mean', 'dewpoint_mean', 'rltv_hum_mean',
       'air_temperature_min', 'rltv_hum_min', 'air_temperature_max',
       'prcp_count', 'mean_consumption', 'median_consumption',
       'std_consumption', 'day', 'holiday', 'month_sin', 'month_cos',
       'day_of_week_sin', 'day_of_week_cos'],
      dtype='object')

In [ ]:
df.head()

,air_temperature_mean,dewpoint_mean,rltv_hum_mean,air_temperature_min,rltv_hum_min,air_temperature_max,prcp_count,mean_consumption,median_consumption,std_consumption,day,holiday,month_sin,month_cos,day_of_week_sin,day_of_week_cos
0,5.925000,5.208333,95.250000,2.4,82.3,10.6,0,7.063833,6.2460,5.373299,2,0,-0.5,0.866025,0.974928,-0.222521
1,9.670833,8.041667,90.112500,6.1,74.5,13.6,2,8.426043,7.4890,6.219456,3,0,-0.5,0.866025,0.433884,-0.900969
2,10.541667,6.891667,78.779167,6.1,59.8,13.0,2,9.350267,8.7385,6.377867,4,0,-0.5,0.866025,-0.433884,-0.900969
3,9.416667,6.333333,81.745833,4.3,66.9,12.9,0,10.255256,9.4770,6.689567,5,0,-0.5,0.866025,-0.974928,-0.222521
4,9.904167,5.004167,73.425000,0.5,46.8,13.9,0,10.946154,9.8030,7.351755,6,0,-0.5,0.866025,-0.781831,0.623490


In [ ]:
# extracting the features only
features = ['air_temperature_mean', 'dewpoint_mean', 'rltv_hum_mean',
       'air_temperature_min', 'rltv_hum_min', 'air_temperature_max',
       'prcp_count', 'holiday', 'month_sin',
       'month_cos', 'day_of_week_sin', 'day_of_week_cos']
features_df = df[features]
features_df.head(3)

,air_temperature_mean,dewpoint_mean,rltv_hum_mean,air_temperature_min,rltv_hum_min,air_temperature_max,prcp_count,holiday,month_sin,month_cos,day_of_week_sin,day_of_week_cos
0,5.925000,5.208333,95.250000,2.4,82.3,10.6,0,0,-0.5,0.866025,0.974928,-0.222521
1,9.670833,8.041667,90.112500,6.1,74.5,13.6,2,0,-0.5,0.866025,0.433884,-0.900969
2,10.541667,6.891667,78.779167,6.1,59.8,13.0,2,0,-0.5,0.866025,-0.433884,-0.900969


In [ ]:
# extracting the targets
target = ['mean_consumption', 'median_consumption',
       'std_consumption']
target_df = df[target]
target_df.head(3)

,mean_consumption,median_consumption,std_consumption
0,7.063833,6.2460,5.373299
1,8.426043,7.4890,6.219456
2,9.350267,8.7385,6.377867


In [ ]:
len(target_df), len(features_df)

(827, 827)

In [ ]:
# experiment with one target only
one_target_df= target_df["mean_consumption"]

# Model Training

In [ ]:
# 1. XGBOOST model

## Single output

In [ ]:
!pip install xgboost


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

model = XGBRegressor(eta = 0.1, gamma = 4)

In [ ]:
def check_model(model, X_train=X_train, y_train=y_train, X_test = X_test, y_test = y_test, features= features_df, target=one_target_df, cv=5):
    cv_scores = cross_val_score(model, features_df, target, cv=5, scoring='neg_mean_squared_error')
    print("Cross-validation scores (negative MSE):", cv_scores)
    print("Mean MSE across all folds:", -cv_scores.mean())
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    plt.scatter(y_test, prediction)
    plt.xlabel("actual")
    plt.ylabel("XGBOOST prediction")
    plt.grid(alpha=0.1)
    plt.show();

In [ ]:
model = XGBRegressor(eta = 0.1, gamma = 4)
check_model(model=model, cv=5)

In [ ]:
# Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()

check_model(model=lr_model)

In [ ]:
# Polynomial Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
lr2_model = LinearRegression()
pr_model = PolynomialFeatures(degree=4)

In [ ]:
X_train_poly = pr_model.fit_transform(X_train)
features_poly = pr_model.fit_transform(features_df)
X_test_poly = pr_model.fit_transform(X_test)
lr2_model = LinearRegression()

# Conclusion

In [ ]:
# The XGBOOST multi output model pereformed well on baseline, which further needs tuning